# 1. Multi-class and Multi-Label Classification Using Support Vector Machines

In [ ]:
!pip install imblearn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

In [2]:
import random
import warnings
warnings.filterwarnings("ignore")
from sklearn import metrics

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

In [4]:
import pandas as pd
import numpy as np

### (a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29. Choose 70% of the data randomly as the training set.

In [6]:
df = pd.read_csv('data/Frogs_MFCCs.csv')

In [7]:
train_data, test_data = train_test_split(df, train_size=0.7, shuffle=True)

### (b) Each instance has three labels: Families, Genus, and Species. Each of the labels has multiple classes. We wish to solve a multi-class and multi-label problem. One of the most important approaches to multi-class classification is to train a classifier for each label. We first try this approach:

### i. Research exact match and hamming score/ loss methods for evaluating multi-label classification and use them in evaluating the classifiers in this problem.

In [8]:
from sklearn.metrics import hamming_loss

In [9]:
def compute_hamming_loss(pred):
    hamming_loss = []
    for l in categories:
        hamming_loss.append(metrics.hamming_loss(test_df[l], pred[l]))
    return round(np.mean(hamming_loss), 2)

In [10]:
df.columns.values

array(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5',
       'MFCCs_ 6', 'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10',
       'MFCCs_11', 'MFCCs_12', 'MFCCs_13', 'MFCCs_14', 'MFCCs_15',
       'MFCCs_16', 'MFCCs_17', 'MFCCs_18', 'MFCCs_19', 'MFCCs_20',
       'MFCCs_21', 'MFCCs_22', 'Family', 'Genus', 'Species', 'RecordID'],
      dtype=object)

In [11]:
categories = ['Family', 'Genus', 'Species']

In [12]:
x_train = train_data.iloc[:, :-4]
x_test = test_data.iloc[:, :-4]
y_train = train_data.iloc[:, -4:-1]
y_test = test_data.iloc[:, -4:-1]

In [13]:
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in categories:
    print('**Processing {}**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train_data[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Hamming Loss is {}'.format(round(hamming_loss(test_data[category], prediction), 2)))
    print('Exact Match score is {}'.format(round(accuracy_score(test_data[category], prediction), 2)))
    print("\n")
    

**Processing Family**
Hamming Loss is 0.08
Exact Match score is 0.92


**Processing Genus**
Hamming Loss is 0.08
Exact Match score is 0.92


**Processing Species**
Hamming Loss is 0.08
Exact Match score is 0.92




### ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both standardized and raw attributes and report the results.

In [14]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold

In [15]:
from sklearn.model_selection import GridSearchCV
parameters = {'C':np.logspace(-5, 8, 10), 
              'gamma': np.append(np.logspace(-4, -1, 10), np.logspace(0, 2, 10))}

In [ ]:
pred = pd.DataFrame()
for category in categories:
    print('**Processing {}**'.format(category))
    svc = SVC(kernel='rbf', decision_function_shape='ovr')
    kf = StratifiedKFold(n_splits=10, shuffle=True)
    clf = GridSearchCV(svc, parameters, cv=kf, scoring='accuracy')
    clf.fit(x_train, train_data[category])
    
    # calculating test accuracy
    sv = clf.best_estimator_
    y_pred = sv.predict(x_test)
    y_true = test_data[category]
    pred[category] = y_pred
    print('Best Parameter {}'.format(clf.best_params_))
    print('Precision {}'.format(round(metrics.precision_score(y_true, y_pred, average='micro'), 2)))
    print('Recall {}'.format(round(metrics.recall_score(y_true, y_pred, average='micro'), 2)))
    print('F1 Score {}'.format(round(metrics.f1_score(y_true, y_pred, average='micro'), 2)))
    print('Exact Match score is {}'.format(round(accuracy_score(y_true, y_pred), 2)))
    print("\n\n")

**Processing Family**
Best Parameter {'gamma': 2.7825594022071245, 'C': 166.81005372000593}
Precision 0.99
Recall 0.99
F1 Score 0.99
Exact Match score is 0.99



**Processing Genus**


In [ ]:
print('Prediction for Test data:\n', pred.head())

In [ ]:
pred = pd.DataFrame(pred)
print("The hamming loss for Gaussian kernel SVM is", compute_hamming_loss(pred))

### iii. Repeat 1(b)ii and L1-penalized SVMs. Remember to standardize the attributes

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [ ]:
pred2 = pd.DataFrame()
for category in categories:
    print('**Processing {}**'.format(category))
    svc = LinearSVC(penalty='l1', multi_class='ovr', dual=False)
    kf = StratifiedKFold(n_splits=10, shuffle=True)
    clf = GridSearchCV(svc, parameters, cv=kf, scoring='accuracy')
    clf.fit(x_train, train_data[category])

    # calculating test accuracy
    y_pred = clf.predict(x_test)
    y_true = test_data[category]
    pred[category] = y_pred
    print('Best Parameter {}'.format(clf.best_params_))
    print('Precision {}'.format(round(metrics.precision_score(y_true, y_pred, average='micro'), 2)))
    print('Recall {}'.format(round(metrics.recall_score(y_true, y_pred, average='micro'), 2)))
    print('F1 Score {}'.format(round(metrics.f1_score(y_true, y_pred, average='micro'), 2)))
    #print('Hamming Loss is {}'.format(round(hamming_loss(y_true, y_pred), 2)))
    print('Exact Match score is {}'.format(round(accuracy_score(y_true, y_pred), 2)))
    print("\n\n")

In [ ]:
print('Prediction for Test data:\n', pred2.head())

In [ ]:
pred2 = pd.DataFrame(pred2)
print("The hamming loss for linear SVM is", compute_hamming_loss(pred2))

### iv. Repeat 1(b)iii by using SMOTE or any other method you know to remedy class imbalance. Report your conclusions about the classifiers you trained.

In [ ]:
sm = SMOTE(random_state=2)

pred2 = pd.DataFrame()
for category in categories:
    print('**Processing {}**'.format(category))
    svc = LinearSVC(penalty='l1', multi_class='ovr', dual=False)
    kf = StratifiedKFold(n_splits=10, shuffle=True)
    clf = GridSearchCV(svc, parameters, cv=kf, scoring='accuracy')
    x_train_res, y_train_res = sm.fit_sample(x_train, train_data[category])
    clf.fit(x_train_res, y_train_res)

    # calculating test accuracy
    y_pred = clf.predict(x_test)
    y_true = test_data[category]
    pred[category] = y_pred
    print('Best Parameter {}'.format(clf.best_params_))
    print('Precision {}'.format(round(metrics.precision_score(y_true, y_pred, average='micro'), 2)))
    print('Recall {}'.format(round(metrics.recall_score(y_true, y_pred, average='micro'), 2)))
    print('F1 Score {}'.format(round(metrics.f1_score(y_true, y_pred, average='micro'), 2)))
    #print('Hamming Loss is {}'.format(round(hamming_loss(y_true, y_pred), 2)))
    print('Exact Match score is {}'.format(round(accuracy_score(y_true, y_pred), 2)))
    print("\n\n")

### v. Extra Practice: Study the Classifier Chain method and apply it to the above problem.

In [ ]:
!pip install skmultilearn

In [ ]:
from sklearn.multioutput import ClassifierChain

# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.multioutput import ClassifierChain
svc = LinearSVC(penalty='l1', multi_class='ovr', dual=False)

y_tr = pd.DataFrame()
for i in range(3):
  d = dict(zip(y_train.iloc[:, i], range(len(y_train.iloc[:, i].unique()))))
  y_tr[i] = y_train.iloc[:, i].map(d)
  
chains = [ClassifierChain(svc, order='random', random_state=5)
          for i in range(10)]
for chain in chains:
    chain.fit(x_train, y_train)

Y_pred_chains = np.array([chain.predict(x_test) for chain in
                          chains])
chain_jaccard_scores = [jaccard_similarity_score(y_test, Y_pred_chain >= .5)
                        for Y_pred_chain in Y_pred_chains]

print(Y_pred_chains.mean(axis=0))

In [ ]:
y_train.iloc[:,0].unique()

### vi. Extra Practice: Research how confusion matrices, precision, recall, ROC, and AUC are defined for multi-label classification and compute them for the classifiers you trained in above.

# 2. K-Means Clustering on a Multi-Class and Multi-Label Data Set

## Monte-Carlo Simulation: Perform the following procedures 50 times, and report the average and standard deviation of the 50 Hamming Distances that you calculate.

### (a) Use k-means clustering on the whole Anuran Calls (MFCCs) Data Set (do not split the data into train and test, as we are not performing supervised learning in this exercise). Choose k automatically based on one of the methods provided in the slides (CH or Gap Statistics or scree plots or Silhouettes) or any other method you know.

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

In [ ]:
from tqdm import tqdm
from statistics import mode

In [ ]:
# clustering dataset
# determine k using elbow method
# k means determine k
K = range(2,15)
ks = []
dist = []
blabels = []
hamming_losses = []

for i in tqdm(range(50)):
    diss = []
    silh = []
    results = []
    X = df.iloc[:, :-4]
    for k in range(2, 15):
        kmeanModel = KMeans(n_clusters=k).fit(X)
        label = kmeanModel.labels_
        results.append(label)
        silh.append(silhouette_score(X, label))
        diss.append(sum(np.min(cdist(X, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / X.shape[0])

    index = np.argmax(silh)
    best_k = index + 2
    ks.append(best_k)
    dist.append(diss)
    best_labels = results[index]
    blabels.append(best_labels)
    print('The best value of K is', best_k)
    
    
    labels_df = df[['Family', 'Genus', 'Species']].copy()
    labels_df['kmeans_label'] = best_labels
    majority_label = {}
    for l in range(best_k):
        cluster = labels_df[labels_df['kmeans_label'] == l]
        majority_triplet = {}
        for tl in ['Family', 'Genus', 'Species']:
            majority_triplet[tl] = cluster[tl].value_counts().idxmax()
        majority_label[l] = majority_triplet
    print("Majority labels:", majority_label)

    #compute hamming loss
    
    misclassifed = 0
    for l in range(best_k):
        cluster = labels_df[labels_df['kmeans_label'] == l]
        for tl in ['Family', 'Genus', 'Species']:
            misclassifed += sum(cluster[tl] != majority_label[l][tl])
    hamming_loss = misclassifed / (len(df) * 3)
    hamming_losses.append(hamming_loss)

kk = np.argmax(mode(ks)) + 2
print('The values of k is', mode(ks))
print('Best Labels are', blabels[kk])
print('Mean of Hamming Loss', np.mean(hamming_losses))
print('Std of Hamming Loss', np.std(hamming_losses))

distortions = []
for i in range(len(dist)):
    av = 0
    for j in range(len(dist)):
         av += dist[j][i]
    distortions.append(av/len(dist))

# Plot the elbow
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

### (b) In each cluster, determine which family is the majority by reading the true labels. Repeat for genus and species.

### (c) Now for each cluster you have a majority label triplet (family, genus, species). Calculate the average Hamming distance (score) between the true labels and the labels assigned by clusters.